Langchain Reference for Text Summarization:
https://python.langchain.com/docs/tutorials/summarization/

In [77]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [78]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027D59A489B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027D59A499D0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [79]:
from langchain.schema import(
    AIMessage, 
    HumanMessage,
    SystemMessage
)

In [80]:
speech="""
People across the country, involved in government, political, and social activities, are dedicating their time to make the ‘Viksit Bharat Sankalp Yatra’ (Developed India Resolution Journey) successful. Therefore, as a Member of Parliament, it was my responsibility to also contribute my time to this program. So, today, I have come here just as a Member of Parliament and your ‘sevak’, ready to participate in this program, much like you.

In our country, governments have come and gone, numerous schemes have been formulated, discussions have taken place, and big promises have been made. However, my experience and observations led me to believe that the most critical aspect that requires attention is ensuring that the government’s plans reach the intended beneficiaries without any hassles. If there is a ‘Pradhan Mantri Awas Yojana’ (Prime Minister’s housing scheme), then those who are living in jhuggis and slums should get their houses. And he should not need to make rounds of the government offices for this purpose. The government should reach him. Since you have assigned this responsibility to me, about four crore families have got their ‘pucca’ houses. However, I have encountered cases where someone is left out of the government benefits. Therefore, I have decided to tour the country again, to listen to people’s experiences with government schemes, to understand whether they received the intended benefits, and to ensure that the programs are reaching everyone as planned without paying any bribes. We will get the real picture if we visit them again. Therefore, this ‘Viksit Bharat Sankalp Yatra’ is, in a way, my own examination. I want to hear from you and the people across the country whether what I envisioned and the work I have been doing aligns with reality and whether it has reached those for whom it was meant.

It is crucial to check whether the work that was supposed to happen has indeed taken place. I recently met some individuals who utilized the Ayushman card to get treatment for serious illnesses. One person met with a severe accident, and after using the card, he could afford the necessary operation, and now he is recovering well. When I asked him, he said: “How could I afford this treatment? Now that there is the Ayushman card, I mustered courage and underwent an operation. Now I am perfectly fine.”  Such stories are blessings to me.

The bureaucrats, who prepare good schemes, expedite the paperwork and even allocate funds, also feel satisfied that 50 or 100 people who were supposed to get the funds have got it. The funds meant for a thousand villages have been released. But their job satisfaction peaks when they hear that their work has directly impacted someone’s life positively. When they see the tangible results of their efforts, their enthusiasm multiplies. They feel satisfied. Therefore, ‘Viksit Bharat Sankalp Yatra’ has had a positive impact on government officers. It has made them more enthusiastic about their work, especially when they witness the tangible benefits reaching the people. Officers now feel satisfied with their work, saying, “I made a good plan, I created a file, and the intended beneficiaries received the benefits.” When they find that the money has reached a poor widow under the Jeevan Jyoti scheme and it was a great help to her during her crisis, they realise that they have done a good job. When a government officer listens to such stories, he feels very satisfied.

There are very few who understand the power and impact of the ‘Viksit Bharat Sankalp Yatra’. When I hear people connected to bureaucratic circles talking about it, expressing their satisfaction, it resonates with me. I’ve heard stories where someone suddenly received 2 lakh rupees after the death of her husband, and a sister mentioned how the arrival of gas in her home transformed her lives. The most significant aspect is when someone says that the line between rich and poor has vanished. While the slogan ‘Garibi Hatao’ (Remove Poverty) is one thing, but the real change happens when a person says, “As soon as the gas stove came to my house, the distinction between poverty and affluence disappeared.
"""

In [81]:
chat_message = [
    SystemMessage(content="You are expert with summarizing speeches."),
    HumanMessage(content=f"Please provide a short and concise summary of the following speech\n Text:{speech}")
]

In [82]:
# number of tokens in the speech
llm.get_num_tokens(speech)

909

In [83]:
# This is one way to do so but it will have challenges when we have big content in the chat message
ai_response = llm(chat_message)
print(ai_response)

content='This speech emphasizes the importance of ensuring government schemes effectively reach their intended beneficiaries. The speaker, a Member of Parliament, describes their "Viksit Bharat Sankalp Yatra" (Developed India Resolution Journey) as a personal examination to assess the impact of these programs. Through anecdotes about individuals who benefited from schemes like the PM Awas Yojana and Ayushman Bharat, the speaker highlights the tangible difference these initiatives make in people\'s lives. \n\nThe speaker also underscores the positive impact this journey has had on government officials, who gain satisfaction from witnessing the direct benefits their work provides to the public.  Ultimately, the speech celebrates the transformative power of these programs in bridging the gap between rich and poor and achieving the goal of "Garibi Hatao" (Remove Poverty). \n\n\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 159, 'prompt_tokens': 883, 'total_

In [84]:
print(ai_response.content)

This speech emphasizes the importance of ensuring government schemes effectively reach their intended beneficiaries. The speaker, a Member of Parliament, describes their "Viksit Bharat Sankalp Yatra" (Developed India Resolution Journey) as a personal examination to assess the impact of these programs. Through anecdotes about individuals who benefited from schemes like the PM Awas Yojana and Ayushman Bharat, the speaker highlights the tangible difference these initiatives make in people's lives. 

The speaker also underscores the positive impact this journey has had on government officials, who gain satisfaction from witnessing the direct benefits their work provides to the public.  Ultimately, the speech celebrates the transformative power of these programs in bridging the gap between rich and poor and achieving the goal of "Garibi Hatao" (Remove Poverty). 





### Prompt Template Text Summarization

In [85]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

genericTemplate = """
Write a summary of the following speech:
Speech:{speech}
Translate the precise summary to {language}
"""

prompt=PromptTemplate(
    input_variables=['speech','language'],
    template=genericTemplate
)
prompt

PromptTemplate(input_variables=['language', 'speech'], input_types={}, partial_variables={}, template='\nWrite a summary of the following speech:\nSpeech:{speech}\nTranslate the precise summary to {language}\n')

In [86]:
# Just to Check how much token are there with PromptTemplate
complete_prompt = prompt.format(speech=speech, language="French")
llm.get_num_tokens(complete_prompt)

931

In [87]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

llm_chain = LLMChain(llm=llm, prompt=prompt, output_parser=parser)
response = llm_chain.run({'speech':speech, 'language':'Hindi'})
response
# output = response | parser

"इस भाषण में, एक सांसद ने 'विकसित भारत संकल्प यात्रा' को लेकर अपनी बात रखी।  उनके अनुसार, सरकार के कई योजनाएं हैं, लेकिन जरूरी है कि ये योजनाएं लाभार्थियों तक आसानी से पहुँचें। उन्होंने 'प्रधानमंत्री आवास योजना' का उदाहरण देते हुए बताया कि हर व्यक्ति को जो आवास मिलना चाहिए, वह मिलना चाहिए, बिना किसी परेशानी के। \n\nउन्होंने यात्रा के माध्यम से जनता से सीधे जुड़कर अपने अनुभवों को समझने और सुनिश्चित करने का संकल्प लिया कि सभी योजनाओं का लाभ सभी तक पहुँच रहा है। \n\nसामाजिक सुरक्षा योजना 'आयुष्मान' के बारे में उन्होंने उदाहरण दिया, जहाँ एक व्यक्ति गंभीर दुर्घटना के बाद अपने इलाज के लिए आयुष्मान कार्ड का उपयोग करके अपना जीवन बचा सका।  \n\nयह यात्रा न केवल लोगों को लाभ पहुंचा रही है, बल्कि सरकार के अधिकारियों को भी संतुष्टि दे रही है।  अधिकारी जब देखते हैं कि उनकी योजनाएं लोगों के जीवन में वास्तविक बदलाव ला रही हैं, तो वे प्रेरित होते हैं और अपनी सेवा में समर्पित हो जाते हैं।  \n\nसांसद ने 'विकसित भारत संकल्प यात्रा' को जनता और सरकार के बीच एक पुल के रूप में बताया, जो गरीबी रेखा के दोनों ओर

## Using `Stuff Document Chain` for Text Summarization
- Stuff is the simplest chain type in LangChain for summarization.

- It loads all text content at once, “stuffs” it into a single prompt, and sends it to the LLM.

- Best for small documents or PDFs that fit within the context window of the model.

- It is fast and efficient, but not scalable to large documents.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Load and split the PDF content into LangChain Document chunks
loader = PyPDFLoader("apjspeech.pdf")
# We do so to convert the PDF into manageable document chunks
docs = loader.load_and_split()
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [ ]:
# Create a summarization + translation prompt template
template = """
Write a short and concise summary of the following speech:
Speech:{text}
Translate the precise summary to {language}
"""

# Define the prompt template with two input variables - speech text and target language
prompt = PromptTemplate(
    input_variables=['text','language'],
    template=template
)
prompt

PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template='\nWrite a short and concise summary of the following speech:\nSpeech:{text}\nTranslate the precise summary to {language}\n')

In [ ]:
from langchain.chains.summarize import load_summarize_chain

# Load the 'stuff' summarization chain with the prompt and LLM
# We use "stuff" to concatenate all documents into one single prompt
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt, verbose=True)
# Run the summarization chain with the input documents and desired language
summary = chain.run({'input_documents': docs, 'language': 'Hindi'})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a short and concise summary of the following speech:
Speech:A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online inte

In [91]:
print(summary)

## ए.पी.जे. अब्दुल कलाम के विदाई भाषण का संक्षिप्त सारांश:

यह भाषण भारत के राष्ट्रपति के रूप में अपने कार्यकाल के अंत पर दिया गया था। कलाम ने अपने कार्यकाल के दौरान मिले अनुभवों को साझा किया और देश के विकास के लिए महत्वपूर्ण संदेश दिए। 

**मुख्य बिंदु:**

* **युवाओं की महत्वाकांक्षा:** 2020 तक भारत को विकसित राष्ट्र बनाने का युवाओं का सपना महत्वपूर्ण है।
* **गांवों का सशक्तिकरण:** सभी गाँवों को आर्थिक और भौतिक रूप से जोड़कर ग्रामीण जीवन को बेहतर बनाया जाना चाहिए।
* **कृषि में सुधार:** "बीज से भोजन" के सिद्धांत पर काम करके कृषि उत्पादन को दोगुना करना और पर्यावरण के अनुकूल कृषि प्रणाली को बढ़ावा देना।
* **संघर्ष का सामना:**  कलाम ने प्रेरणादायक कहानियों के माध्यम से  संकटों का सामना करने और उन्हें चुनौती के रूप में देखने पर जोर दिया।
* **सहयोग और संचार:** आपदाओं से निपटने,  रोगियों को दूरस्थ स्वास्थ्य सेवा प्रदान करने और देश के विभिन्न हिस्सों को जोड़ने के लिए तकनीक का उपयोग करना।
* **राष्ट्र की रक्षा:**  ब्रह्मांड और समुद्र में अपने देश की सीमाओं की रक्षा करने वाले सैनिकों, नाविकों और 

## Using Map Reduce for Summarization

In [92]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load_and_split()
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [93]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
)
docs = text_splitter.split_documents(docs)

docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [ ]:
# Define the prompt for summarizing individual chunks of the document (MAP step)
chunk_prompt = """
Summarize the below speech:
Speech:`{text}`
"""

# Create a prompt template that will be applied to each document chunk
# This is the MAP prompt used to summarize small parts of the overall input
map_prompt_template = PromptTemplate(
    input_variables=['text','language'],
    template=chunk_prompt
)

# Define the final prompt for combining all individual summaries (REDUCE step)
# It asks the model to write a structured final summary and then translate it
final_prompt = """
Write a short and concise summary of the entire speech:
Add a title to the summary.
Start with an Introduction and then write the summary in a structured format.
Speech:{text}
Translate the precise summary to {language}
"""

# Create a prompt template for the reduce step (combining all chunk summaries)
final_prompt_template = PromptTemplate(
    input_variables=['text','language'],
    template=final_prompt
)

In [ ]:
from langchain.chains.summarize import load_summarize_chain
# Load the summarization chain with the 'map_reduce' strategy
# - MAP: applies the first prompt to each chunk individually
# - REDUCE: combines all the partial summaries using the final prompt
# map_prompt - Used for individual chunk summarization
# combine_prompt - Used for combining partial summaries
summary_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

# Run the summarization pipeline with the documents and target language
# The LLM will first summarize each chunk, then combine and translate
output = summary_chain.run({'input_documents': docs, 'language': 'Hindi'})



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Summarize the below speech:
Speech:`A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many uniq

In [96]:
print(output)

## ए.पी.जे. अब्दुल कलाम के प्रेरणादायक भाषण का सारांश

**परिचय:**

यह सारांश भारत के पूर्व राष्ट्रपति ए.पी.जे. अब्दुल कलाम के एक प्रेरणादायक भाषण का संक्षिप्त और संक्षिप्त रूप प्रस्तुत करता है। 

**भाषण का सार:**

*  **युवाओं की आकांक्षा:** कलाम ने युवाओं की आकांक्षाओं को भारत की प्रगति के लिए महत्वपूर्ण बताया, जिसके लिए उन्होंने 2020 तक भारत को विकसित राष्ट्र बनाने के लक्ष्य को दोहराया।
*  **गांवों का सशक्तिकरण:** उन्होंने गांवों के सशक्तिकरण पर जोर दिया, विशेष रूप से "बीज से भोजन" (Seed to Food)  मॉडल के माध्यम से कृषि विकास को बढ़ावा देने पर।
*  **साझेदारी और दृढ़ता:** कलाम ने समस्याओं का समाधान साझेदारी के माध्यम से करने और आपदाओं के सामने साहस दिखाने पर बल दिया।  उन्होंने राष्ट्रीय सुरक्षा और सामाजिक परिवर्तन के लिए कनेक्टिविटी के महत्व को भी रेखांकित किया।
*  **युवा आंदोलन:** उन्होंने 2020 तक विकसित भारत के लक्ष्य को प्राप्त करने के लिए युवाओं को समर्पित आंदोलन में शामिल होने का आह्वान किया।

*  **पेरियार PURA मॉडल:** कलाम ने पेरियार PURA (Providing Urban Amenities in Rural Areas

## Using Refine Chain for Summarization

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
)
docs = text_splitter.split_documents(docs)

docs

In [ ]:
# Load the refine summarization chain
# - This chain works by first generating an initial summary from the first document chunk.
# - Then, it sequentially refines that summary by incorporating information from each following chunk.
# - Useful for generating high-quality, context-aware summaries especially when document flow matters.
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    verbose=True
)

output = chain.run({"input_documents": docs})



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many

In [108]:
print(output)

In his departing speech, A P J Abdul Kalam reflects on his five years as President of India, expressing gratitude for the experiences shared with people from all walks of life. He highlights ten key messages from his interactions, emphasizing the pivotal role of youth aspirations in India's development.  

Kalam illustrates this point with an anecdote about a young girl, Anukriti, who questioned him about India's timeline for becoming a developed nation. Touched by her question, Kalam pledged to advocate for her dream at the highest levels of government, recognizing the widespread desire among India's 540 million youth to live in a prosperous, safe, and proud nation.  

He stresses the importance of several key areas:

* **Empowering villages:** Kalam cites a visit to Nagaland, where he witnessed the positive impact of an empowered village council, underscoring the potential for transformation across India's 600,000 villages with proper connectivity and support. He uses the example of 